# PowerShell: Property Selection (Aliases)

NOTE: Be sure to run each performance test example 2 or 3 times to get an average.

What if, for some unknown reason, you need to return a set of files and their associated properties in the following form:

* Name
* Path
* DateCreated
* DateModified

You could use customize the output using the [Select-Object](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.utility/select-object) cmdlet, or you could use a PSObject ([pscustomobject]) to override the default output. Let's compare the performance of each method using a set of files from a typical Windows machine.  We'll use the [Get-ChildItem](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.management/get-childitem) to enumerate .XML [File](https://learn.microsoft.com/en-us/dotnet/api/system.io.file) objects in the C:\Windows\System32 folder. Run the following code block to prepare for the remainder of this notebook.

In [ ]:
$files = Get-ChildItem -Path "c:\windows\system32" -Filter "*.xml" -File
Write-Host "You are ready to proceed!" -ForegroundColor Yellow

In [ ]:
$files[0] | Select-Object *

There's plenty of information available in this sample file output (above). The ```Name``` property is fine as-is. But we need to convert ```FullName``` to ```Path```, ```CreationTime``` to ```DateCreated```, and ```LastWriteTime``` to ```DateModified```.

The ```Select-Object``` cmdlet allows for overriding the propert names and values returned from an object. You use the "Label" and "Expression" parameters to specify the output property name, and associated value, respectively. The example below translates the ```FullName``` property to ```Path```:

In [ ]:
$files[0] | Select-Object @{label='Path';expression={$_.FullName}}

You can also shorten "Label" to just "L" or "l", and "Expression" to "E" or "e". The tricky part is the formatting and making sure not to end up with unbalanced braces.

```@{ Label="Name"; Expression={"Value"}}```

"Value" in the above example can be an explicit value, or a variable, property or function. Anything that can return a value.

Run the following to see how to convert the built-in properties names to the desired output names:

In [ ]:
$files[0] | Select-Object @{l='Name';e={$_.Name}},@{l='Path';e={$_.FullName}},@{l='DateModified';e={$_.LastWriteTime}},@{l='DateCreated';e={$_.CreationTime}}

Next, we will use a PSObject to modify the output

In [ ]:
$files[0] | Foreach-Object {
	[pscustomobject]@{
		Name = $_.Name
		Path = $_.FullName
		DateCreated = $_.CreationTime
		DateModified = $_.LastWriteTime
	}
}

Same result for each. But now we can test which is processed faster.

## Performance Tests: Which is Faster?

### TEST 1 - Select-Object with Custom Labels

In [ ]:
Measure-Command {
	$files | Select-Object @{l='Name';e={$_.Name}},@{l='Path';e={$_.FullName}},@{l='DateModified';e={$_.LastWriteTime}},@{l='DateCreated';e={$_.CreationTime}}
} | Select-Object -ExpandProperty TotalMilliseconds

### TEST 2 - PSObject / [pscustomobject]

In [ ]:
Measure-Command {
	foreach ($file in $files) {
		[pscustomobject]@{
			Name = $_.Name
			Path = $_.FullName
			DateModified = $_.LastWriteTime
			DateCreated = $_.CreationTime
		}
	}
} | Select-Object -ExpandProperty TotalMilliseconds

Another option using a PSObject is to build out the properties using ```Add-Member```. Let's try one example first...

In [ ]:
$f = New-Object -TypeName PSObject
$f | Add-Member -MemberType NoteProperty -Name Name -Value $files[0].Name
$f | Add-Member -MemberType NoteProperty -Name Path -Value $files[0].FullName
$f | Add-Member -MemberType NoteProperty -Name DateCreated -Value $files[0].CreationTime
$f | Add-Member -MemberType NoteProperty -Name DateModified -Value $files[0].LastWriteTime
$f

### TEST 3 - PSObject using Add-Member

In [ ]:
Measure-Command {
	foreach ($file in $files) {
		$f = New-Object -TypeName PSObject
		$f | Add-Member -MemberType NoteProperty -Name Name -Value $file.Name
		$f | Add-Member -MemberType NoteProperty -Name Path -Value $file.FullName
		$f | Add-Member -MemberType NoteProperty -Name DateCreated -Value $file.CreationTime
		$f | Add-Member -MemberType NoteProperty -Name DateModified -Value $file.LastWriteTime
	}
} | Select-Object -ExpandProperty TotalMilliseconds

## More Tests

### TEST 4 - Instantiate Object, then Add Properties

In [ ]:
Measure-Command {
	for ($i = 0; $i -lt 1000; $i++) {
		$item = New-Object -TypeName psobject
		$item | Add-Member -MemberType NoteProperty -Name Caption -Value "My Object"
		$item | Add-Member -MemberType NoteProperty -Name Counter -Value $i
	}
} | Select-Object -ExpandProperty TotalMilliseconds

### TEST 5 - Instantiate Object with Properties

In [ ]:
Measure-Command {
	for ($i = 0; $i -lt 1000; $i++) {
		$item = New-Object -TypeName psobject -Property @{Caption = "My Object"; Counter = $i}
	}
} | Select-Object -ExpandProperty TotalMilliseconds

### TEST 6 - PSObject / [pscustomobject]

In [ ]:
Measure-Command {
	for ($i = 0; $i -lt 1000; $i++) {
		$item = [pscustomobject]@{Caption = "My Object"; Counter = $i}
	}
} | Select-Object -ExpandProperty TotalMilliseconds

## Conclusion

These are very basic examples. You could bend the results of any of these by changing some of the ways you build the custom object. For example, the NoteProperty is one of multiple property [MemberType](https://learn.microsoft.com/en-us/dotnet/api/system.management.automation.psmembertypes) options for the [Add-Member](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.utility/add-member) cmdlet. You may feel this is worrying too much about a small thing. But if you end up processing a large amount of data the performance differences can vary significantly.